In [410]:
import json
import re

import requests
import scrapy

In [411]:
headers = {'User-Agent': 'UNC Journo Class'}

In [412]:
base_url = 'http://goheels.com'
url = base_url + '//roster.aspx?path=wvball'

In [413]:
resp = requests.get(url, headers=headers)

In [414]:
body_str = resp.content.decode('utf-8')

In [415]:
sel = scrapy.Selector(text=body_str)

In [416]:
table = sel.css('table')[0]


In [417]:
cols = table.css('th').xpath('string()').extract()

In [418]:
rows = table.css('tr')[1:]

In [419]:
players = []
for r in rows:
    data = {}
    for i, d in enumerate(r.css('td')):
        a = d.css('a')
        if a:
            t = a.xpath('text()').extract()[0]
            data['href'] = a.xpath('@href').extract()[0]
        else:
            t = d.xpath('text()').extract()
        data[cols[i]] = t
    players.append(data)

In [420]:
#Copy & pasted from the refrence file to get the data... Sorry could not figure it out myself 

In [421]:
def fetch_bio(player):
    player_url = base_url + player['href']
    resp = requests.get(player_url, headers=headers)
    player_txt = resp.content.decode('utf-8')
    sel = scrapy.Selector(text=player_txt)
    player['sel'] = sel
    player['bio'] = sel.css('#sidearm-roster-player-bio').xpath('string()').extract()[0]
    player['img'] = sel.css('.sidearm-roster-player-image img').xpath('@src').extract()[0]

In [422]:
js_obj_rx = re.compile(r'.*?responsive-roster-bio\.ashx.*?(?P<obj>{.*?})')

In [423]:
def fetch_stats(player):
    text = player['sel'].xpath('string()').extract()[0]
    parts = text.split('$.getJSON("/services/')[1:]
    captured = js_obj_rx.findall(''.join(parts))
    clean_objs = []
    for obj_str in captured:
        # We only want the stats object...
        if 'stats' not in obj_str:
            continue

        obj_str = obj_str.replace('{', '').replace('}', '')
        obj_str = obj_str.replace("'", '').replace('"', '')
        obj_pairs = obj_str.split(',')
        obj_pairs = [x.split(":") for x in obj_pairs]
        clean_pairs = []
        for pair in obj_pairs:
            clean_pairs.append(['"{}"'.format(p.strip()) for p in pair])
        colonized = [":".join(p) for p in clean_pairs]
        commas = ','.join(colonized)
        json_str = "{" + commas + "}"
        clean_objs.append(json.loads(json_str))
    
    player['stats_url'] = stats_url = (
        "http://goheels.com/services/responsive-roster-bio.ashx?"
        "type={type}&rp_id={rp_id}&path={path}&year={year}"
        "&player_id={player_id}"
    ).format(**clean_objs[0])

    resp = requests.get(stats_url, headers=headers)
    json_stats = json.loads(resp.content.decode("utf-8"))
    player['raw_stats'] = json_stats

In [424]:
for p in players:
    fetch_bio(p)
    fetch_stats(p)

In [425]:
txt = p['raw_stats']['career_stats']
sel = scrapy.Selector(text=txt)

In [426]:
#Finished Bio! move to he the next table...

In [427]:
#Trying get player data

In [428]:
def get_name(player):
        return player['Full Name']

In [429]:
def get_data_1(player):
        col_name =[]
        txt = player['raw_stats']['career_stats']
        sel = scrapy.Selector(text=txt)
        table = sel.css('.sidearm-table')
        for col in (table.css('tr')[1].css('td')):
            t = col.xpath('string()').extract()[0]
            col_name.append(t)
        return col_name

In [430]:
txt = players[0]['raw_stats']['career_stats']
sel = scrapy.Selector(text=txt)
table = sel.css('.sidearm-table')
table.css('tr')[1].css('td').xpath('string()').extract()[0]

'122'

In [431]:
get_col(players[0])

['122',
 '33',
 '12',
 '0.10',
 '1',
 '23',
 '.478',
 '503',
 '4.12',
 '12',
 '0.10',
 '26']

In [432]:
def get_data_2(player):
        col_name =[]
        txt = player['raw_stats']['career_stats']
        sel = scrapy.Selector(text=txt)
        table = sel.css('.sidearm-table')
        for col in (table.css('tr')[4].css('td')):
            t = col.xpath('string()').extract()[0]
            col_name.append(t)
        return col_name
        return chart_col


In [433]:
txt = players[0]['raw_stats']['career_stats']
sel = scrapy.Selector(text=txt)
table = sel.css('.sidearm-table')
table.css('tr')[4].css('td').xpath('string()').extract()

['168', '1.38', '0', '0', '2', '2', '0.02', '0', '27', '25.0', '0.20']

In [434]:
get_col_2(players[0])

['Season',
 'DIG',
 'D/S',
 'RE',
 'BS',
 'BA',
 'TB',
 'B/S',
 'BE',
 'BHE',
 'PTS',
 'PTS/S']

In [466]:
def get_col_1(player):
    col_data = []
    txt = player['raw_stats']['career_stats']
    sel = scrapy.Selector(text=txt)
    table = sel.css('.sidearm-table')
    for col in (table.css('tr')[0].css('th')):
            t = col.xpath('string()').extract()[0]
            col_data.append(t)            
    col_data.pop(0)
    return col_data

In [467]:
get_col_1(players[0])

['SP', 'MP', 'K', 'K/S', 'E', 'TA', '%', 'A', 'A/S', 'SA', 'SA/S', 'SE']

In [470]:
def get_col_2(player):
    txt = player['raw_stats']['career_stats']
    sel = scrapy.Selector(text=txt)
    table = sel.css('.sidearm-table')
    chart_data=table.css('tr')[3].css('th').xpath('string()').extract()
    chart_data.pop(0)
    return chart_data

In [471]:
get_col_2(players[0])


['DIG', 'D/S', 'RE', 'BS', 'BA', 'TB', 'B/S', 'BE', 'BHE', 'PTS', 'PTS/S']

In [472]:
def get_chart_title_1(player):
    txt = player['raw_stats']['career_stats']
    sel = scrapy.Selector(text=txt)
    caption = sel.css('.hide')
    title = caption.css('caption').xpath('string()').extract()[0]
    return title
    

In [473]:
get_chart_title_1 (players[5])

'Career Offensive Statistics'

In [474]:
def get_chart_title_2(player):
    txt = player['raw_stats']['career_stats']
    sel = scrapy.Selector(text=txt)
    caption = sel.css('.hide')
    title = caption.css('caption').xpath('string()').extract()[1]
    return title
    

In [475]:
player_stats =[]

In [484]:
def parse_stats(player):
    for raw_key, raw_val in player['raw_stats'].items():
        txt = player['raw_stats'][raw_key]
        if not txt:
            print('Skipping {} for {}'.format(raw_key, player['Full Name']))
            continue
    txt = player['raw_stats']['career_stats']
    sel = scrapy.Selector(text=txt)
    cols = get_col_1(player)
    data = get_data_1(player)
    player_stats.append({
            'name':get_name(player),
            'Chart#1':get_chart_title_1(player),
            'cols_1':get_col_1(player),
            'data_1':get_data_1(player),
            'Chart#2':get_chart_title_2(player),
            'cols_2':get_col_2(player),
            'data_2':get_data_2(player),
        })
    return player_stats
    
        


In [485]:
for p in players:
    parse_stats(p)

Skipping current_stats for Mariah Evans
Skipping gamehigh_stats for Mariah Evans
Skipping current_stats for Keimaya Hunter
Skipping gamehigh_stats for Keimaya Hunter
Skipping current_stats for Raleigh Clark
Skipping career_stats for Raleigh Clark
Skipping gamehigh_stats for Raleigh Clark


ValueError: Selector needs either text or root argument

In [489]:
player_stats[0]

{'Chart#1': 'Career Offensive Statistics',
 'Chart#2': 'Career Defensive Statistics',
 'cols_1': ['DIG',
  'D/S',
  'RE',
  'BS',
  'BA',
  'TB',
  'B/S',
  'BE',
  'BHE',
  'PTS',
  'PTS/S'],
 'data_1': ['122',
  '33',
  '12',
  '0.10',
  '1',
  '23',
  '.478',
  '503',
  '4.12',
  '12',
  '0.10',
  '26'],
 'data_2': ['168',
  '1.38',
  '0',
  '0',
  '2',
  '2',
  '0.02',
  '0',
  '27',
  '25.0',
  '0.20'],
 'name': 'Mariah Evans'}

In [490]:
to_dump = player_stats
with open('scraped_players.json', 'w') as f:
    json.dump(to_dump, f)

In [491]:
cat scraped_players.json | cut -c 1-100

[{"name": "Mariah Evans", "Chart#1": "Career Offensive Statistics", "cols_1": ["DIG", "D/S", "RE", "


In [492]:
to_dump

[{'Chart#1': 'Career Offensive Statistics',
  'Chart#2': 'Career Defensive Statistics',
  'cols_1': ['DIG',
   'D/S',
   'RE',
   'BS',
   'BA',
   'TB',
   'B/S',
   'BE',
   'BHE',
   'PTS',
   'PTS/S'],
  'data_1': ['122',
   '33',
   '12',
   '0.10',
   '1',
   '23',
   '.478',
   '503',
   '4.12',
   '12',
   '0.10',
   '26'],
  'data_2': ['168',
   '1.38',
   '0',
   '0',
   '2',
   '2',
   '0.02',
   '0',
   '27',
   '25.0',
   '0.20'],
  'name': 'Mariah Evans'},
 {'Chart#1': 'Career Offensive Statistics',
  'Chart#2': 'Career Defensive Statistics',
  'cols_1': ['DIG',
   'D/S',
   'RE',
   'BS',
   'BA',
   'TB',
   'B/S',
   'BE',
   'BHE',
   'PTS',
   'PTS/S'],
  'data_1': ['95',
   '26',
   '18',
   '0.19',
   '2',
   '28',
   '.571',
   '394',
   '4.15',
   '6',
   '0.06',
   '25'],
  'data_2': ['146',
   '1.54',
   '1',
   '0',
   '0',
   '0',
   '0.00',
   '2',
   '18',
   '24.0',
   '0.25'],
  'name': 'Kendra Koetter'},
 {'Chart#1': 'Career Offensive Statistics',
  'Cha